- Entrenar un modelo de extracción de entidades y relaciones 

Recordemos que son ideas entrelazadas y antes de pasar a extraer relaciones, primero es necesario pensar en las entidades a las cuales les queremos asignar una relación.

> La idea original es planteada en el siguiente articulo:

https://towardsdatascience.com/how-to-train-a-joint-entities-and-relation-extraction-classifier-using-bert-transformer-with-spacy-49eb08d91b5c

Nosotros hacemos las modificaciones pertinentes para entrenar nuestra propia base de datos en idioma español

# Elementos a instalar

In [ ]:
#Los de extraccion de entidades
!pip install -U spacy==3.2.4
!python -m spacy download es_dep_news_trf
!pip install git+https://github.com/explosion/spacy-transformers
#y unas paqueterias extra
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component
#Suponemos que rel_component se intalo en /content/drive/MyDrive/modelos_estancia/rel_component
#de no ser el caso, solo tenener las consideraciones de donde tenemos la carpeta rel_component

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 45.1 MB/s 
     |████████████████████████████████| 10.1 MB 34.5 MB/s 
     |████████████████████████████████| 457 kB 45.2 MB/s 
     |████████████████████████████████| 660 kB 73.0 MB/s 
     |████████████████████████████████| 42 kB 438 kB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410.2 MB 34 kB/s 
     |████████████████████████████████| 51 kB 130 kB/s 
     |████████████████████████████████| 3.8 MB 10.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 880 kB 41.0 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f361103a53996897c25f2c49712b3e208bff2fc4dddccc77561e03ea3db3d4ea
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-

✔ Cloned 'tutorials/rel_component' from explosion/projects
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


In [ ]:
cd /content/drive/MyDrive/modelos_estancia/rel_component

/content/drive/MyDrive/modelos_estancia/rel_component


> Con ayuda de las siguientes lineas creamos archivos .spacy a partir de archivos json o txt, dichos archivos contienen las entidades asi como las relaciones que utilizamos para entrenar nuestro modelo.


Para poder crear los archivos json, es necesario realizar un etiquetado manual utilizando la herramienta ubiai https://ubiai.tools/, despues lo unico que hacemos es descargar los archivos json y transformarlos en archivos .spacy que utilizaremos para entrenar nuestro modelo.

Los archivos json los descargamos utilizando

In [ ]:
import json
import typer
from pathlib import Path
from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.lang.es import Spanish
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("es")
# Create a blank Tokenizer with just the Spanish vocab
#Crear un tokenizador en blanco solo con el vocabulario español
msg = Printer()
SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "SEÑALA": "SEÑALA",
    "AGREDIO": "AGREDIO",
    "EN": "EN",
    "ES": "ES",
    "CONEXION": "CONEXION"
}


ann = "/content/drive/MyDrive/modelos_estancia/rel_component/rel_test.txt"
train_file='/content/drive/MyDrive/modelos_estancia/rel_component/relations_training.spacy'
dev_file=''
test_file=''

def main(json_loc: Path, train_file: Path, dev_file: Path, test_file: Path):
    """Creating the corpus from the Prodigy annotations."""
    Doc.set_extension("rel", default={},force=True)
    vocab = Vocab()

    docs = {"train": [], "dev": [], "test": [], "total": []}
    ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
    count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
    count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

    with open(json_loc, encoding="utf8") as jsonfile:
        file = json.load(jsonfile)
        for example in file:
            span_starts = set()
            neg = 0
            pos = 0
                    # Parse the tokens
            tokens=nlp(example["document"])    

            spaces=[]
            spaces = [True if tok.whitespace_ else False for tok in tokens]
            words = [t.text for t in tokens]
            doc = Doc(nlp.vocab, words=words, spaces=spaces)


            # Parse the GGP entities
            spans = example["tokens"]
            entities = []
            span_end_to_start = {}
            for span in spans:
                entity = doc.char_span(
                     span["start"], span["end"], label=span["entityLabel"]
                 )

                span_end_to_start[span["token_start"]] = span["token_start"]
                #print(span_end_to_start)
                entities.append(entity)
                span_starts.add(span["token_start"])

            doc.ents = entities

            # Parse the relations
            rels = {}
            for x1 in span_starts:
                for x2 in span_starts:
                    rels[(x1, x2)] = {}
                    #print(rels)
            relations = example["relations"]
            #print(len(relations))
            for relation in relations:
                # the 'head' and 'child' annotations refer to the end token in the span
                # but we want the first token
                start = span_end_to_start[relation["head"]]
                end = span_end_to_start[relation["child"]]
                label = relation["relationLabel"]
                #print(rels[(start, end)])
                #print(label)
                #label = MAP_LABELS[label]
                if label not in rels[(start, end)]:
                    rels[(start, end)][label] = 1.0
                    pos += 1
                    #print(pos)
                    #print(rels[(start, end)])

            # The annotation is complete, so fill in zero's where the data is missing
            for x1 in span_starts:
                for x2 in span_starts:
                    for label in MAP_LABELS.values():
                        if label not in rels[(x1, x2)]:
                            neg += 1
                            rels[(x1, x2)][label] = 0.0

                            #print(rels[(x1, x2)])
            doc._.rel = rels
            #print(doc._.rel)

            # only keeping documents with at least 1 positive case
            if pos > 0:
                    docs["total"].append(doc)
                    count_pos["total"] += pos
                    count_all["total"] += pos + neg

                    
                    
    #print(len(docs["total"]))
    docbin = DocBin(docs=docs["total"], store_user_data=True)
    docbin.to_disk(train_file)
    msg.info(
        f"{len(docs['total'])} training sentences"
    )
main(ann, train_file, dev_file, test_file)

FileNotFoundError: ignored

In [ ]:
# Con los datos de entrenamiento y prueba creados en formato spacy
# creamos una carpeta que tiene por nombre 'data', data se ubica en 'rel_component/data',

# Entrenar el modelo

0. Situarnos en el directorio que tiene por nombre rel_components
1. Modificar el path de train, test y dev en archivos project.yml
2. Podemos cambiar el modelo en configs/ archivos cgf
3. Disminuir max_length en configs/rel_tok2vec.cfg del token predeterminado 100 a 20 
> @misc = "rel_instance_generator.v1"
> max_length = 20

In [ ]:
#Comenzamos el entrenamiento de nuestro modelo 
!spacy project run train_cpu


================================= train_cpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_tok2vec.cfg --output training --paths.train data/relations_train.spacy --paths.dev data/relations_test.spacy -c ./scripts/custom_functions.py
ℹ Saving to output directory: training
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-30 18:04:28,352] [INFO] Set up nlp object from config
[2022-05-30 18:04:28,362] [INFO] Pipeline: ['tok2vec', 'relation_extractor']
[2022-05-30 18:04:28,367] [INFO] Created vocabulary
[2022-05-30 18:04:28,368] [INFO] Finished initializing nlp object
[2022-05-30 18:04:32,676] [INFO] Initialized pipeline components: ['tok2vec', 'relation_extractor']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'relation_extractor']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS RELAT...  REL_MICRO_P 

# Evaluar el modelo

In [ ]:
!spacy project run evaluate


================================== evaluate ==================================
Running command: /usr/bin/python3 ./scripts/evaluate.py training/model-best data/relations_test.spacy False

Random baseline:
threshold 0.00 	 {'rel_micro_p': '4.59', 'rel_micro_r': '100.00', 'rel_micro_f': '8.77'}
threshold 0.05 	 {'rel_micro_p': '4.51', 'rel_micro_r': '93.75', 'rel_micro_f': '8.61'}
threshold 0.10 	 {'rel_micro_p': '4.54', 'rel_micro_r': '89.06', 'rel_micro_f': '8.63'}
threshold 0.20 	 {'rel_micro_p': '4.64', 'rel_micro_r': '81.25', 'rel_micro_f': '8.78'}
threshold 0.30 	 {'rel_micro_p': '4.71', 'rel_micro_r': '71.09', 'rel_micro_f': '8.83'}
threshold 0.40 	 {'rel_micro_p': '4.56', 'rel_micro_r': '59.38', 'rel_micro_f': '8.47'}
threshold 0.50 	 {'rel_micro_p': '3.77', 'rel_micro_r': '39.84', 'rel_micro_f': '6.88'}
threshold 0.60 	 {'rel_micro_p': '3.72', 'rel_micro_r': '31.25', 'rel_micro_f': '6.65'}
threshold 0.70 	 {'rel_micro_p': '3.68', 'rel_micro_r': '22.66', 'rel_micro_f': '6.34'}
t

> Para este punto es necesario mover los archivos rel_pipe y rel_model a la carpeta principal rel_model, ya que realizaremos la importación de algunas librerias

---



In [ ]:
cd /content/drive/MyDrive/modelos_estancia/rel_component

/content/drive/MyDrive/modelos_estancia/rel_component


In [ ]:
#Cargamos nuestro modelo entrenado para identificar entidades
#Vease Jupiter Entrenar un modelo de extracción de entidades
import spacy
nlp = spacy.load("/content/drive/MyDrive/modelos_estancia/model-best", disable = ['parser', 'tagger'])
nlp.add_pipe('sentencizer')

In [ ]:
####Cargamos los elementos necesarios para realizar la extracción de entidades
####rel_pipe.py y rel_model.py deben moverse o copiarse de la carpeta scripts a 
####la carpeta rel_component donde es el directorio actual 
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [ ]:
####Cargamos nuestro modelo de extracción de relaciones
####que debio de guardarse en la carpeta de nombre training ubicada en la carpeta rel_component
nlp2 = spacy.load("/content/drive/MyDrive/modelos_estancia/rel_component/training/model-best")

In [ ]:
text = [
'''
El intercambio sobre los datos de homicidios que mantuvieron el periodista Jorge Ramos y el presidente de México, Andrés Manuel López Obrador, el mes pasado, derivó hacia una importante discusión sobre la libertad de expresión, pero a costa de opacar el tema de fondo: México atraviesa el periodo más violento de su historia moderna.
La inseguridad ocasionada por el crimen organizado, y que persiste en varias regiones del país, es el principal problema para la producción minera, reconoció Fracisco Quiroga, titular de Minería en la Secretaría de Economía.
Las empresas mineras que operan en el país enfrentan inseguridad e ilícitos como los robos de mercancía y la extorsión, por lo que el gobierno federal trabaja con la fuerza pública de los distintos niveles para enfrentarla, dijo el subsecretario de Minería de la Secretaría de Economía, Francisco Quiroga
La inseguridad causada por el crimen organizado, que persiste en varias regiones del país, es el principal problema para la producción minera, admitió Francisco Quiroga, subsecretario de Minería.
El subsecretario de Minería de la Secretaría de Economía (SE), Francisco Quiroga Fernández, dijo que el sector enfrenta delitos como robos y extorsión, sin embargo, ya trabajan en abatir esta problemática.
'''
]

In [ ]:
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
    print([(ent.text, ent.label_) for ent in doc.ents])


[('Jorge Ramos', 'PERSONAS'), ('presidente', 'PUESTOSDETRABAJO'), ('Andrés Manuel López Obrador', 'PERSONAS'), ('crimen organizado', 'OTROS'), ('Fracisco Quiroga', 'PERSONAS'), ('titular', 'PUESTOSDETRABAJO'), ('Secretaría de Economía', 'ORGANIZACIONES'), ('subsecretario', 'PUESTOSDETRABAJO'), ('Secretaría de Economía', 'ORGANIZACIONES'), ('Francisco Quiroga', 'PERSONAS'), ('crimen organizado', 'OTROS'), ('Francisco Quiroga', 'PERSONAS'), ('subsecretario', 'PUESTOSDETRABAJO'), ('subsecretario de Minería de la Secretaría de Economía (SE)', 'PUESTOSDETRABAJO'), ('Francisco Quiroga Fernández', 'PERSONAS')]


In [ ]:
##Tomamos las entidades generadas del modelo reconocimiento de entidades 
##y son las entradas a al modelo de relaciones 
for name, proc in nlp2.pipeline[:]:
          print(1)
          doc = proc(doc)

1
1


In [ ]:
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                if rel_dict['AGREDIO'] >=0.005 :
                  print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")

In [ ]:
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                if rel_dict['SEÑALA'] >=0.005 :
                  print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")

 entities: ('Francisco Quiroga', 'crimen organizado') --> predicted relation: {'SEÑALA': 0.14460193, 'AGREDIO': 0.00024804816, 'EN': 2.0611535e-09, 'ES': 0.005892892, 'CONEXION': 8.046362e-08}


In [ ]:
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                if rel_dict['CONEXION'] >=0.0005 :
                  print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")

 entities: ('Jorge Ramos', 'Andrés Manuel López Obrador') --> predicted relation: {'SEÑALA': 1.13628175e-05, 'AGREDIO': 3.111034e-05, 'EN': 2.0611535e-09, 'ES': 0.001975511, 'CONEXION': 0.22109523}
 entities: ('Secretaría de Economía', 'Fracisco Quiroga') --> predicted relation: {'SEÑALA': 4.284274e-07, 'AGREDIO': 0.0001659334, 'EN': 2.0611535e-09, 'ES': 1.2524188e-07, 'CONEXION': 0.0008763566}
 entities: ('Secretaría de Economía', 'subsecretario') --> predicted relation: {'SEÑALA': 1.890733e-08, 'AGREDIO': 8.529301e-07, 'EN': 2.0611535e-09, 'ES': 0.003582913, 'CONEXION': 0.0008183951}


In [ ]:
for value, rel_dict in doc._.rel.items():
        for sent in doc.sents:
          for e in sent.ents:
            for b in sent.ents:
              if e.start == value[0] and b.start == value[1]:
                if rel_dict['ES'] >=0.5 :
                  print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")

 entities: ('Andrés Manuel López Obrador', 'presidente') --> predicted relation: {'SEÑALA': 8.9756696e-07, 'AGREDIO': 5.536075e-09, 'EN': 2.0611535e-09, 'ES': 0.9997402, 'CONEXION': 2.0611535e-09}
 entities: ('Fracisco Quiroga', 'titular') --> predicted relation: {'SEÑALA': 0.0003780298, 'AGREDIO': 4.0036937e-07, 'EN': 7.880128e-08, 'ES': 0.51306117, 'CONEXION': 1.6992441e-08}
 entities: ('Francisco Quiroga', 'subsecretario') --> predicted relation: {'SEÑALA': 6.406767e-07, 'AGREDIO': 5.8838407e-07, 'EN': 2.0611535e-09, 'ES': 0.9989825, 'CONEXION': 7.231341e-07}
 entities: ('Francisco Quiroga', 'subsecretario') --> predicted relation: {'SEÑALA': 2.375778e-07, 'AGREDIO': 7.183829e-06, 'EN': 2.0611535e-09, 'ES': 0.99987257, 'CONEXION': 1.9590213e-06}
